# Sentencing

In [ ]:
# This Python file uses the following encoding: utf-8
import os
from lib.words import not_so_naive_split, replace_evil_dots_and_underscores, tokenize_doc
from lib.parallel_run import parallel_run, parallelize
import numpy as np
import codecs

Divisione del testo in frasi, rimozione di stop words, numeri, punteggiatura...

In [ ]:
# Esempio di split

t = not_so_naive_split(text, split='.')
#t

In [ ]:
# old, no utf-8...
def sentence_splitting(src_dir, out_dir, digit_replacement='NUM', split='.', min_words=2, max_n=-1):
    i = 0
    for f in os.listdir(src_dir):
        txt = open(src_dir+'/'+f).read()
        splitted = not_so_naive_split(txt, digit_replacement, split, min_words)
        with open(out_dir+'/'+f, 'w') as of:
            for l in splitted:
                of.write(' '.join(l)+'\n')
        if max_n>0 and i >= max_n:
            break
        i += 1
        
def sentence_tokenize_file(f, src_dir, out_dir, min_words=2):
    txt = open(src_dir+'/'+f).read()
    splitted = tokenize_doc(txt, min_words)
    with codecs.open(out_dir+'/'+f, 'w', 'utf-8') as of:
        for l in splitted:
            of.write(u' '.join(l)+u'\n')
        
def sentence_tokenization(src_dir, out_dir, min_words=2):
    parallelize(sentence_tokenize_file)(os.listdir(src_dir), src_dir, out_dir, min_words)

In [ ]:
# Per atti costitutivi
#sentence_splitting('../atti_costitutivi_txt','../atti_costitutivi_sentences')
sentence_tokenization('../atti_costitutivi_txt','../atti_costitutivi_sentences')

In [ ]:
# Per atti non costitutivi
#sentence_splitting('../atti_non_costitutivi_txt2','../atti_non_costitutivi_sentences')
sentence_tokenization('../atti_non_costitutivi_txt','../atti_non_costitutivi_sentences')

In [ ]:
# Per verbali
sentence_tokenization('../verbali_a05_txt/','../verbali_a05_sentences/')

In [ ]:
# Per atti non costitutivi scansionati
sentence_splitting('../ocr/scanned_non_costitutivi','../atti_non_costitutivi_sentences')

In [ ]:
# Esempio di testo elaborato

text = open('../atti_non_costitutivi_sentences/'+'4578843770001.txt').read()
print text

## Conversione in CSV

In [ ]:
def folders_to_csv(csv_file, dirs, labels):
    '''
    Writes to a csv file the lines contained in a list of directories,
    assuming each directory is associated to one label'''
    with codecs.open(csv_file,'w', 'utf-8') as out_file:
        out_file.write('filename,sentence,label\n')
        for d, label in zip(dirs, labels):
            for f in os.listdir(d):
                with codecs.open(d+u'/'+f, 'r', 'utf-8') as in_file:
                    for line in in_file:
                        out_file.write((u','.join([f[:-4],line.strip(),label]))+u'\n')
 

In [ ]:
folders_to_csv('../atti2.csv', ['../atti_costitutivi_sentences/','../atti_non_costitutivi_sentences/','../verbali_a05_sentences/'],['costitutivo','non_costitutivo','non_costitutivo'])

## Statistiche PANDAS

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def print_column_stats(col):
    print 'Mean', col.mean()
    print 'Stddev', col.std()
    print 'Min', col.min()
    print 'Max', col.max()

In [ ]:
df = pd.read_csv("../atti2.csv", encoding='utf-8')
#df

In [ ]:
df

In [ ]:
df_nc = df.loc[df['label'] == 'non_costitutivo']
df_c = df.loc[df['label'] == 'costitutivo']

## Distribuzione numero parole per linea

In [ ]:
s_len = df['sentence'].apply(lambda s:len(s.split()))

In [ ]:
print_column_stats(s_len)

In [ ]:
plt.figure(figsize=(20,10))
plt.hist([s for s in s_len if s<200], bins=100, normed=True) #[s for s in s_len if s<200]
#plt.yscale('log', nonposy='clip')
plt.show()

In [ ]:
s_len_c = df_c['sentence'].apply(lambda s:len(s.split()))
s_len_nc = df_nc['sentence'].apply(lambda s:len(s.split()))

In [ ]:
def hist_len(s_len):
    plt.figure(figsize=(20,10))
    plt.hist([s for s in s_len if s<200], bins=100, normed=True)
    #plt.yscale('log', nonposy='clip')
    plt.show()

In [ ]:
hist_len(s_len_c)
hist_len(s_len_nc)

## Distribuzione numero linee per file

In [ ]:
lines_per_file = df[['filename','sentence']].groupby(['filename']).size()

In [ ]:
print_column_stats(lines_per_file)

In [ ]:
plt.figure(figsize=(20,10))
plt.hist([l for l in lines_per_file if l<500], bins=300)
#plt.yscale('log', nonposy='clip')
plt.show()

In [ ]:
lines_per_file_nc = df_nc[['filename','sentence']].groupby(['filename']).size()
lines_per_file_c = df_c[['filename','sentence']].groupby(['filename']).size()

In [ ]:
print_column_stats(lines_per_file_c)
print_column_stats(lines_per_file_nc)

In [ ]:
def hist_lines(lines):
    plt.figure(figsize=(20,10))
    plt.hist([l for l in lines if l<300], bins=300)
    #plt.yscale('log', nonposy='clip')
    plt.show()

In [ ]:
hist_lines(lines_per_file_c)
hist_lines(lines_per_file_nc)

## NLTK test

In [1]:
from nltk import word_tokenize, pos_tag, ne_chunk
import codecs
from lib import words as wd



In [11]:
text = open('../statuti/molesti/roman_stat_in_mezzo.txt').read()
sents = wd.sentences_doc(text)
tree = ne_chunk(pos_tag(word_tokenize(sents[0])))

UnicodeEncodeError: 'ascii' codec can't encode character u'\xe0' in position 6: ordinal not in range(128)

## Experiments on statuto indexes

In [1]:
import re
import codecs
import operator
import bisect
from lib import article_numbers as an
from lib import words as wd

In [2]:
#text = codecs.open('../atti_costitutivi_txt/4957502190001.txt', encoding="utf-8").read()
#text = codecs.open('../statuti/molesti/4909480860001.txt', encoding="utf-8").read()
#text = codecs.open('../statuti/molesti/4908140980001.txt', encoding="utf-8").read()
#text = codecs.open('../statuti/molesti/4907851720001.txt', encoding="utf-8").read()
#text = codecs.open('../statuti/molesti/4907874650001.txt', encoding="utf-8").read() #roman
text = codecs.open('../statuti/molesti/roman_stat_in_mezzo.txt', encoding="utf-8").read() #roman inside
#text = codecs.open('../statuti/molesti/schifo.txt', encoding="utf-8").read()


In [3]:
lines = [l.strip() for l in text.split('\n') if len(l.strip())>0]

In [4]:
list(enumerate(lines))

[(0, u'Dr. ALBA MARIA FERRARA'),
 (1, u'NOTAIO'),
 (2, u'20122 Milano - Via Visconti di Modrone n. 2'),
 (3, u'tel. 02/ 86.92.090 - 86.08.73 - 86.90.691'),
 (4, u'Fax 02/86454529'),
 (5, u'Repertorio n. 102483'),
 (6, u'Raccolta n. 19374'),
 (7, u"COSTITUZIONE DI SOCIETA' A RESPONSABILITA' LIMITATA"),
 (8, u'Repubblica Italiana'),
 (9, u"- L'anno duemiladiciassette, il giorno ventisei del mese di gennaio"),
 (10, u'26 gennaio 2017'),
 (11, u'- In Milano, nel mio studio posto in Via Visconti di Modrone n. 2.'),
 (12,
  u'- Davanti a me, Dr. Alba Maria Ferrara, Notaio in Milano, iscritta presso il'),
 (13, u'Collegio Notarile di Milano,'),
 (14, u'sono presenti i Signori:'),
 (15,
  u'- Cornegliani Mario, nato a Como (CO) il giorno 20 luglio 1946, domiciliato a'),
 (16,
  u'Milano (MI), Via Santa Radegonda n. 8, il quale interviene al presente atto nella'),
 (17, u'sua veste di Procuratore della Societ\xe0 "TORNO CONSULTING S.R.L.",'),
 (18,
  u'societ\xe0 con unico socio, con sede legal

In [5]:
an.end_statuto_text(text)

u'Art. 18) Le decisioni in ordine alle materie di cui ai punti 4., 5., 6., 7. del'

In [6]:
def text_idx_lines_idx(idx, lines):
    prev_idx = 0
    cur_idx = 0
    i_lines = 0
    while cur_idx < idx and i_lines < len(lines):
        prev_idx = cur_idx
        cur_idx += len(lines[i_lines]) + 1
        i_lines += 1
    return i_lines

'''
def num_before(l, n):
    prev = 0
    for e in l:
        if e < n:
            prev = e
        else:
            return prev
'''
        
def num_before(l, n):
    if len(l)==0:
        return 0
    i_res = bisect.bisect_left(l,n)-1
    if i_res == -1:
        return 0
    return l[i_res]
        
def end_statuto_init(lines, matches, init_idx):
    sent_idx_before = num_before([i for i,e in matches], init_idx)
    idx_to_find = -1
    for i,e in matches:
        if i == sent_idx_before:
            idx_to_find = e + 1
            break
    
    im = iter(matches)
    _, t = next(im)
    
    for i,e in matches:
        if e == idx_to_find and e != t + 1:
            return lines[i]
        t = e
    return ''

def end_statuto_init_text(text, init_statuto_str):
    lines = [l.strip() for l in text.split('\n') if len(l.strip())>0]
    matches = an.matches_list(lines)
    corrected_matches = an.filter_matches_errors(matches)
    
    i_text = text.index(init_statuto_str)
    #print(i_text)
    il = text_idx_lines_idx(i_text, lines)
    
    return end_statuto_init(lines, corrected_matches, il)

In [7]:
begin_statuto = ['S T A T U T O', 'STATUTO', 'STATUTO DELLA SOCITET', 'FUNZIONAMENTO DELLA SOCIET', 'PATTI DISCIPLINANTI']

for b in begin_statuto:
    i = wd.index_ignore_whitespaces(text, b)
    if i > -1:
        break
print(i)
ti = text[i:i+100]
print(ti)

2635
STATUTO
DENOMINAZIONE - OGGETTO - SEDE - DURATA
Art. 1) E' costituita una Società a responsabilità l


In [8]:
il = an.text_idx_lines_idx(i, lines)

In [9]:
il

61

In [10]:
ml = an.matches_list(lines)

In [11]:
end_statuto_init(lines, ml, il)

u"Art. IV - L'amministrazione della Societ\xe0 viene affidata ad un Amministratore"

In [13]:
an.end_statuto_init_text(text, ti)

u"Art. IV - L'amministrazione della Societ\xe0 viene affidata ad un Amministratore"

# Random tests section

In [1]:
from lib.utils import download_from_storage_if_not_present

In [2]:
download_from_storage_if_not_present('infocamere-poc', 'gs://demo-dataset/ocr-01/02628680346_2081942240001.pdf', 'tmp/tmp.pdf')